In [1]:
import os
# nvidia-smi
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import pandas as pd
import csv
import numpy as np
from tqdm import tqdm
tqdm.pandas()

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
PREFIX = "../data/"
PA_PATH = PREFIX + "sap2017-connotation-frames-power-agency/"
PP_PATH = PREFIX + "ma2020-powertransformer/con_rew_data/para/"

# read data

Paraphrase corpus:

@inproceedings{creutz-2018-open,
    title = "Open Subtitles Paraphrase Corpus for Six Languages",
    author = "Creutz, Mathias",
    booktitle = "Proceedings of the Eleventh International Conference on Language Resources and Evaluation ({LREC} 2018)",
    month = may,
    year = "2018",
    address = "Miyazaki, Japan",
    publisher = "European Language Resources Association (ELRA)",
    url = "https://aclanthology.org/L18-1218",
}

We use the data as preprocessed by ma-etal-2020-powertransformer 

@inproceedings{ma-etal-2020-powertransformer,
    title = "{P}ower{T}ransformer: Unsupervised Controllable Revision for Biased Language Correction",
    author = "Ma, Xinyao  and
      Sap, Maarten  and
      Rashkin, Hannah  and
      Choi, Yejin",
    booktitle = "Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing (EMNLP)",
    month = nov,
    year = "2020",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2020.emnlp-main.602",
    doi = "10.18653/v1/2020.emnlp-main.602",
    pages = "7426--7441",
}

Get data at https://github.com/michellesima/powerTransformer

In [5]:
dev = pd.read_csv(PP_PATH + 'dev.csv', index_col=0)

In [6]:
train = pd.read_csv(PP_PATH + 'train.csv', index_col=0)

In [7]:
dev.head()

,orisen,oridel,oricat,oriverbs,parasen,paradel,paracat,paraverbs
0,"Let 's see your hands , girls .","<VERB> 's see your hands , girls .",equal,let,Let me see your hand .,<VERB> me see your hand .,equal,let
1,You 'll pay for this .,You 'll <VERB> for this .,equal,pay,You 're gonna pay for this .,You 're <VERB> na pay for this .,equal,go
2,What 's going on back there ?,What 's <VERB> on back there ?,equal,go,What 's going on over there ?,What 's <VERB> on over there ?,equal,go
3,Everything is gonna be all right .,Everything is <VERB> na be all right .,equal,go,We 're gonna be fine .,We 're <VERB> na be fine .,equal,go
4,I 'm gonna throw up .,I ' m <VERB> na throw up .,equal,go,I am going to puke .,I am <VERB> to puke .,equal,go


In [8]:
train.head()

,orisen,oridel,oricat,oriverbs,parasen,paradel,paracat,paraverbs
0,Get away from him .,<VERB> away from him .,pos,get,Let go of him .,Let <VERB> of him .,equal,go
1,That 's all I want to do .,That 's all I want to <VERB> .,pos,do,This is all I want .,This is all I <VERB> .,neg,want
2,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean
3,Why are you stopping ?,Why are you <VERB> ?,pos,stop,Why have we stopped ?,Why <VERB> we stopped ?,neg,have
4,I won 't let you .,I won ' t <VERB> you .,equal,let,l won 't allow it .,l won ' t <VERB> it .,pos,allow


In [9]:
dev = dev.rename(columns={"oricat": "oriagency", "paracat": "paraagency"})
train = train.rename(columns={"oricat": "oriagency", "paracat": "paraagency"})

# power

In [10]:
pa = pd.read_csv(PA_PATH + "agency_power_prepro.csv", sep=',')

In [11]:
def get_power(verb):
    try:
        return pa[(pa.lemma == verb.strip()) & (~pa.prep.notna())].power.values[0]
    except:
        print(verb)

In [12]:
dev['oripower'] = dev.oriverbs.progress_apply(get_power)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2364.06it/s]


In [13]:
train['oripower'] = train.oriverbs.progress_apply(get_power)

100%|██████████████████████████████████████████████████████████████████████████| 45000/45000 [00:19<00:00, 2350.48it/s]


In [14]:
dev['parapower'] = dev.paraverbs.progress_apply(get_power)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2318.25it/s]


In [15]:
train['parapower'] = train.paraverbs.progress_apply(get_power)

100%|██████████████████████████████████████████████████████████████████████████| 45000/45000 [00:19<00:00, 2339.91it/s]


In [16]:
dev.head()

,orisen,oridel,oriagency,oriverbs,parasen,paradel,paraagency,paraverbs,oripower,parapower
0,"Let 's see your hands , girls .","<VERB> 's see your hands , girls .",equal,let,Let me see your hand .,<VERB> me see your hand .,equal,let,power_agent,power_agent
1,You 'll pay for this .,You 'll <VERB> for this .,equal,pay,You 're gonna pay for this .,You 're <VERB> na pay for this .,equal,go,power_agent,power_equal
2,What 's going on back there ?,What 's <VERB> on back there ?,equal,go,What 's going on over there ?,What 's <VERB> on over there ?,equal,go,power_equal,power_equal
3,Everything is gonna be all right .,Everything is <VERB> na be all right .,equal,go,We 're gonna be fine .,We 're <VERB> na be fine .,equal,go,power_equal,power_equal
4,I 'm gonna throw up .,I ' m <VERB> na throw up .,equal,go,I am going to puke .,I am <VERB> to puke .,equal,go,power_equal,power_equal


In [17]:
train.head()

,orisen,oridel,oriagency,oriverbs,parasen,paradel,paraagency,paraverbs,oripower,parapower
0,Get away from him .,<VERB> away from him .,pos,get,Let go of him .,Let <VERB> of him .,equal,go,power_agent,power_equal
1,That 's all I want to do .,That 's all I want to <VERB> .,pos,do,This is all I want .,This is all I <VERB> .,neg,want,power_agent,power_theme
2,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,power_equal,power_equal
3,Why are you stopping ?,Why are you <VERB> ?,pos,stop,Why have we stopped ?,Why <VERB> we stopped ?,neg,have,power_agent,power_agent
4,I won 't let you .,I won ' t <VERB> you .,equal,let,l won 't allow it .,l won ' t <VERB> it .,pos,allow,power_agent,power_agent


In [18]:
total = train.append(dev)

In [20]:
total = total.reset_index()

In [23]:
total = total.drop('index', axis=1)

In [25]:
total

,orisen,oridel,oriagency,oriverbs,parasen,paradel,paraagency,paraverbs,oripower,parapower
0,Get away from him .,<VERB> away from him .,pos,get,Let go of him .,Let <VERB> of him .,equal,go,power_agent,power_equal
1,That 's all I want to do .,That 's all I want to <VERB> .,pos,do,This is all I want .,This is all I <VERB> .,neg,want,power_agent,power_theme
2,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,power_equal,power_equal
3,Why are you stopping ?,Why are you <VERB> ?,pos,stop,Why have we stopped ?,Why <VERB> we stopped ?,neg,have,power_agent,power_agent
4,I won 't let you .,I won ' t <VERB> you .,equal,let,l won 't allow it .,l won ' t <VERB> it .,pos,allow,power_agent,power_agent
...,...,...,...,...,...,...,...,...,...,...
54995,I said shut up .,I <VERB> shut up .,pos,say,I said shut up .,I <VERB> shut up .,pos,say,power_equal,power_equal
54996,I have a little confession to make .,I <VERB> a little confession to make .,neg,have,I have a little confession to make .,I <VERB> a little confession to make .,neg,have,power_agent,power_agent
54997,I wouldn 't go that far .,I wouldn ' t <VERB> that far .,equal,go,I wouldn 't go that far .,I wouldn ' t <VERB> that far .,equal,go,power_equal,power_equal
54998,I 've just got to .,I ' ve just <VERB> to .,pos,get,I 've just got to .,I ' ve just <VERB> to .,pos,get,power_agent,power_agent


In [26]:
total.to_csv('../../../Daten/ma2020-powertransformer/' + '/pa-transformer/para_sentences.csv')

## map power and agency

In [27]:
total = pd.read_csv('../../../Daten/ma2020-powertransformer/' + '/pa-transformer/para_sentences.csv', index_col=0)

In [28]:
total.head()

,orisen,oridel,oriagency,oriverbs,parasen,paradel,paraagency,paraverbs,oripower,parapower
0,Get away from him .,<VERB> away from him .,pos,get,Let go of him .,Let <VERB> of him .,equal,go,power_agent,power_equal
1,That 's all I want to do .,That 's all I want to <VERB> .,pos,do,This is all I want .,This is all I <VERB> .,neg,want,power_agent,power_theme
2,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,power_equal,power_equal
3,Why are you stopping ?,Why are you <VERB> ?,pos,stop,Why have we stopped ?,Why <VERB> we stopped ?,neg,have,power_agent,power_agent
4,I won 't let you .,I won ' t <VERB> you .,equal,let,l won 't allow it .,l won ' t <VERB> it .,pos,allow,power_agent,power_agent


In [29]:
total.oripower = total.oripower.map({'power_agent': 'pos', 'power_equal': 'equal', 'power_theme': 'neg' })

In [30]:
total.parapower = total.parapower.map({'power_agent': 'pos', 'power_equal': 'equal', 'power_theme': 'neg' })

In [33]:
# power must be given
total = total[(total.oripower.notna() & total.parapower.notna())]

In [35]:
total.head()

,orisen,oridel,oriagency,oriverbs,parasen,paradel,paraagency,paraverbs,oripower,parapower
0,Get away from him .,<VERB> away from him .,pos,get,Let go of him .,Let <VERB> of him .,equal,go,pos,equal
1,That 's all I want to do .,That 's all I want to <VERB> .,pos,do,This is all I want .,This is all I <VERB> .,neg,want,pos,neg
2,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,What 's that suppose to mean ?,What 's that suppose to <VERB> ?,neg,mean,equal,equal
3,Why are you stopping ?,Why are you <VERB> ?,pos,stop,Why have we stopped ?,Why <VERB> we stopped ?,neg,have,pos,pos
4,I won 't let you .,I won ' t <VERB> you .,equal,let,l won 't allow it .,l won ' t <VERB> it .,pos,allow,pos,pos


In [36]:
total.to_csv('../../../Daten/ma2020-powertransformer/' + '/pa-transformer/para_train.csv')